# Proyecto 1. Calendario laLiga

#### Partidos

In [2]:
import pandas as pd
from itertools import permutations

df = pd.read_csv('../Data/equipos_data.csv', index_col = 0)
equipos = list(df['equipo'].unique())
partidos = list(permutations(df['equipo'].unique(), 2))

#### Días

In [3]:
import pickle
filename = '../Data/list_calendar.pkl'
lista_dias = open(filename, 'rb')
with open(filename, 'rb') as f:
    lista_dias=pickle.load(f)
# lista_dias

#### Finde

In [4]:
import pickle
filename = '../Data/list_day.pkl'
es_finde = open(filename, 'rb')
with open(filename, 'rb') as f:
    es_finde=pickle.load(f)
# Es finde? 1 si, 0 no

### Modelo

In [5]:
filename = '../Data/predicciones.pickle'
predicciones = open(filename, 'rb')
with open(filename, 'rb') as f:
    predicciones=pickle.load(f)

## Modelado

### Variables

https://developers.google.com/optimization/assignment/assignment_example

In [6]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP') #CBC, SCIP, SAT

x = {}
for i in partidos:
    for j in lista_dias:
        x[i, j] = solver.BoolVar(f'Partido {i} jugado el dia {j}')

### Variables ficticias

In [7]:
dl = {}
for i in partidos: 
    dl[i] = solver.IntVar(0, len(lista_dias)-1, f'Día lectivo en el que se juega {i}')
    solver.Add(dl[i] == solver.Sum([dia * x[i,j] for dia, j in enumerate(lista_dias)]))

### Restricciones

<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [8]:
for j in lista_dias:
    solver.Add(solver.Sum([x[i, j] for i in partidos])<=4)

<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [10]:
for i in partidos:
    solver.Add(solver.Sum([x[i, j] for j in lista_dias])==1)

<div class="alert alert-info">
  <strong>Las cadenas de TV prefieren que los partidos sean el finde</strong>
</div>

In [11]:
solver.Add(solver.Sum([x[i, lista_dias[j]] for j in range(len(lista_dias)) if es_finde[j] for i in partidos])>=0.6*380)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001C00B519F00> >

<div class="alert alert-info">
  <strong>Separar los partidos de ida y vuelta (mínimo 60 días jugables)</strong>
</div>

In [ ]:
for i in partidos:
    for indice, j in enumerate(lista_dias[:len(lista_dias)-60+1]):
        solver.Add(solver.Sum([x[i, j1]+x[(i[1],i[0]), j1] for j1 in lista_dias[indice:indice+60+1]])<=1)

<div class="alert alert-info">
  <strong>Es deseable que no se jueguen demasiados partidos seguidos en casa o como visitante</strong>
</div>

In [12]:
for equipo1 in equipos:
    for dia in range(1, len(lista_dias)+1):
        restriccion = solver.Constraint(-1, 1, '')
        for j in lista_dias[:dia]:
            for equipo2 in equipos:
                if equipo1 != equipo2:
                    restriccion.SetCoefficient(x[(equipo1, equipo2), j], 1)
                    restriccion.SetCoefficient(x[(equipo2, equipo1), j], -1)

### Función objetivo

In [13]:
objective_terms = []
for i in partidos:
    for j in lista_dias:
        objective_terms.append(predicciones[i,j] * x[i, j])
solver.Maximize(solver.Sum(objective_terms))

### Resultado

In [ ]:
from time import time

t0 = time()

solver.SetTimeLimit(45 * 60 * 1000) # Maximo de tiempo en ms
status = solver.Solve()

t1 = time() - t0

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Audiencia total = {int(solver.Objective().Value())} espectadores\n')
    print(f'Audiencia media = {int(solver.Objective().Value()/380)} espectadores\n')
    cont = 0
    for i in partidos:
        for j in lista_dias:
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                cont+=1
                print(f'Partido {i} jugado el día {j}.' +
                      f' Audiencia esperada: {int(predicciones[(i,j)])} espectadores')
                
    print(f'\nPartidos totales jugados: {cont}')
    
else:
    print('Sin solución.')

In [ ]:
print(f'Ha tardado {t1 // 3600} horas, {t1 % 3600 // 60} minutos y {round(t1 % 60, 2)} segundos')

### Comprobación de restricciones

"results" es una matriz en el que las filas son los partidos y las columnas los día. results[i,j] = 1 si se juega el partido i el día j. Como es una matriz no se puede indexar con texto, por lo que "i" y "j" son el índice que corresponde a ese día y a es partido. Por ejemplo, el i = 0 corresponde al partido ('Real Madrid CF', 'FC Barcelona'). Si tienes el nombre del partido o el día y lo quieres pasar a índice puedes utilizar "partido_a_ind" o "dia_a_ind".

In [ ]:
import pandas as pd
import numpy as np

dia_a_ind = {dia : i for i, dia in enumerate(lista_dias)}
partido_a_ind = {p : i for i, p in enumerate(partidos)}
results = np.zeros((len(partidos), len(lista_dias)))

for k,v in x.items():
    i, j = k
    i = partido_a_ind[i]
    j = dia_a_ind[j]
    
    results[i, j] = v.solution_value()
    
# int(round(v.solution_value()))

<div class="alert alert-info">
  <strong>Día lectivo</strong>
</div>

In [ ]:
bien = True
for partido, dia_lectivo in dl.items():
    i = partido_a_ind[partido]
    j = round(dia_lectivo.solution_value())

    if results[i, j] < 0.5:
        print('La restricción no va bien')
        bien = False
        break
    
if bien:
    print('La restricción funciona correctamente')

<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [ ]:
suma = np.sum(results, axis = 0) 
if np.all(suma <= 4):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [ ]:
suma = np.sum(results, axis = 1)
if np.all(suma > 0):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

<div class="alert alert-info">
  <strong>Las cadenas de TV prefieren que los partidos sean el finde</strong>
</div>

In [ ]:
prop = np.sum(results)*0.6
finde = np.array(es_finde) == True
partidos_por_dias = np.sum(results,axis=0)

if np.sum(partidos_por_dias[finde]) >= prop:
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

<div class="alert alert-info">
  <strong>Separar los partidos de ida y vuelta (mínimo 60 días jugables)</strong>
</div>

In [ ]:
for partido in partidos:
    ida = results[partido_a_ind[partido]]
    vuelta = results[partido_a_ind[partido[1], partido[0]]]
    dia_ida = np.where(ida == 1)[0][0]
    dia_vuelta = np.where(vuelta == 1)[0][0]
    if abs(dia_ida - dia_vuelta) < 60:
        print('La restricción no va bien')
else:
    print('La restricción funciona correctamente')

<div class="alert alert-info">
  <strong>Es deseable que no se jueguen demasiados partidos seguidos en casa o como visitante</strong>
</div>

In [ ]:
try:
    for equipo1 in equipos:
        partidos_equipo1 = np.zeros(len(lista_dias))
        for equipo2 in equipos:
            if equipo1 != equipo2:
                local = partido_a_ind[equipo1, equipo2]
                vis = partido_a_ind[equipo2, equipo1]
                partidos_equipo1 += results[local] - results[vis]
        
        aux = 0
        for p in partidos_equipo1:
            aux += p
            if aux < -1 or aux > 1:
                raise RuntimeError("No va")
                
except RuntimeError:
    print('La restricción no va bien')
    
else:
    print('La restricción funciona correctamente')